In [ ]:
import numpy as np
import pandas as pd
# import matplotlib
# matplotlib.use('TkAgg')
import math
import matplotlib.pyplot as plt


path = '/Users/jiayun/PycharmProjects/untitled/untitled/' \
       'untitled/rbot Project/ my code/machine learning/w' \
       'eek3 logistic regression/ex2data2.txt'

data = pd.read_csv(path, header = None, names=['test 1', 'test 2', 'Accepted'])
data.head()

positive = data[data['Accepted'].isin([1])]
negative = data[data['Accepted'].isin([0])]

fig,ax = plt.subplots(figsize = (12,8))
ax.scatter(positive['test 1'], positive['test 2'],s=50, c ='b',marker='o',label='Accepted')
ax.scatter(negative['test 1'], negative['test 2'],s=50, c ='r',marker='*',label='Not Accepted')
ax.legend()
ax.set_xlabel('Test 1 Score')
ax.set_ylabel('Test 2 Score')
plt.show()


'''1.polynomial feature'''
################################################################################
########---------------------1.构建polynomial feature---------------------------########
################################################################################

degree = 5
x1 = data['test 1']
x2 = data['test 2']
data.insert(3, 'ones', 1)

for i in range(1,degree):
    for j in range(0,i):
        data['F' + str(i) + str(j)] = np.power(x1, i-j) * np.power(x2, j) # 这行是给data 一个新列，列名是'F i j '每次计算一次，给一个新的列

'''   下面是对上面一段代码的解读：哈哈哈
     i 是range（1，5）， 就是 1，2，3，4；
        j是每次（0，i），就是从 0 到 i-1；
            i-j 就是每次

for p in range(1,5):
    print(p)
    
for i in range(1,degree):
    for j in range(0,i):
        print(i,j,i-j)

'''
data.drop('test 1', axis=1, inplace = True)         # 把test1 这列删除
data.drop('test 2', axis=1, inplace = True)         # 把test2 这列删除
data.head()

################################################################################
########--------------------polynomial feature end------------------------########
################################################################################



'''2.正则化代价函数'''
################################################################################
########--------------------regularized cost（正则化代价函数） ------------------------########
################################################################################
def sigmoid(z):
    result = 1 / (1+ np.exp(-z))    # 这地方也可以用 np.exp(-z) 一样的
    return  result

def costFunc(theta, x, y, lamda): # lamda 是 learning rate

    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    z = x * (theta.T)

    first_term = np.multiply(-y, np.log(sigmoid(z)))
    second_term = np.multiply((1-y), np.log(1-sigmoid(z)))
    reg = lamda * (np.power(theta[:,1:theta.shape[1]],2))  # np.square(theta[:,1:theta.shape[1]])

    Jval = (np.sum(first_term - second_term)) / (len(x))  + ((np.sum(reg)) / (2*len(x)))

    return Jval

################################################################################
########--------------------regularized cost（正则化代价函数） ------------------------########
################################################################################

'''3.gradient function'''
##############################################################################
########--------------------gradient function------------------------########
################################################################################

def gradient(theta, x, y, lamda):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    z = x * theta.T

    parameter = int(theta.ravel().shape[1])
    m = x.shape[0]

    grad = np.zeros(parameter)
    error = sigmoid(z) - y

    for j in range(parameter):

        term = np.multiply(error, x[:,j])

        if j == 0:
            grad[j] = np.sum(term) / m

        else:
            grad[j] = lamda * np.sum(term) / m

    return grad

##############################################################################
########--------------------gradient function end------------------------########
################################################################################

'''5。计算部分，调用上面的函数'''
##############################################################################
########--------------------计算------------------------########
##############################################################################

cols = data.shape[1]
x = data.iloc[:,1:cols]
y = data.iloc[:,0:1]

x = np.array(x)
y = np.array(y)
theta = np.zeros(x.shape[1])

lamda = 1
costFunc(theta,x,y,lamda)  # test costFunction, make sure they can work
gradient(theta,x,y,lamda)   # test gradient , make sure they can work


import scipy.optimize as opt
#  这是上一次饮用这个 result = opt.fmin_tnc(func=costFunction, x0=theta, fprime=gradient, args=(X,y))
result = opt.fmin_tnc(func=costFunc, x0 = theta, fprime = gradient, args = (x,y,lamda))

theta_final = np.matrix(result[0])
''' prediction function'''
def predict(theta, x):
    # theta = np.matrix(theta)
    # X = np.matrix(X)
    probability = sigmoid(x * theta.T)
    prediction = [ 1 if a >= 0.5 else 0 for a in probability]   # prediction结果是一个list
    return prediction

y_prediction = predict(theta_final,x)
correct = [1 if  ((a==1) and (b==1)) or ((a==0)and (b==0)) else 0 for (a,b) in zip(y, y_prediction)]
accuracy = (sum(map(int, correct)) % len(correct))
print('accuracy = {0}%'.format(accuracy))

##############################################################################
########--------------------计算 end------------------------########
################################################################################

'''7。 scikit learn 取代上面自己写的代码'''
##############################################################################
########--------------------用sikit learn------------------------########
################################################################################
from sklearn import linear_model

model = linear_model.LogisticRegression(penalty='l2', C=1.0)
model.fit(x, y.ravel())


model.fit(x,y.ravel())
model.score(x,y)


##############################################################################
########--------------------sikit learn------------------------########
################################################################################

